<a href="https://colab.research.google.com/github/EyaRIAHI-BE/Soft-Computing-Projects/blob/main/Genetic_Algorithm_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikit-fuzzy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.8/920.8 kB 22.3 MB/s eta 0:00:00


In [2]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

Schéma du régulateur flou initial

In [3]:
def fitness(temp_ext, temp_int):

    #INITIALISATION DES VARIABLES FLUOES (ENTRÉES / SORTIE)

    temp_ext_fuzzy = ctrl.Antecedent(np.arange(0, 51, 1), 'temp_ext')
    temp_int_fuzzy = ctrl.Antecedent(np.arange(30, 42.1, 0.1), 'temp_int')
    puissance = ctrl.Consequent(np.arange(0, 101, 1), 'puissance')

    # DÉFINITION DES FONCTIONS D'APPARTENANCE

    # Température externe
    temp_ext_fuzzy['froid'] = fuzz.trapmf(temp_ext_fuzzy.universe, [0, 0, 10, 15])
    temp_ext_fuzzy['chaud'] = fuzz.trapmf(temp_ext_fuzzy.universe, [25, 30, 50, 50])

    # Température interne
    temp_int_fuzzy['hypothermie'] = fuzz.trapmf(temp_int_fuzzy.universe, [30, 30, 35.5, 36.5])
    temp_int_fuzzy['normal'] = fuzz.trimf(temp_int_fuzzy.universe, [35, 37, 38])
    temp_int_fuzzy['fievre'] = fuzz.trapmf(temp_int_fuzzy.universe, [37.5, 38.5, 42, 42])

    # Puissance de chauffage
    puissance['faible'] = fuzz.trimf(puissance.universe, [0, 15, 30])
    puissance['moyenne'] = fuzz.trimf(puissance.universe, [30, 50, 70])
    puissance['maximale'] = fuzz.trimf(puissance.universe, [70, 85, 100])

    #  CONSTRUCTION DES RÈGLES FLOUES

    rule1 = ctrl.Rule(temp_int_fuzzy['hypothermie'] & temp_ext_fuzzy['froid'], puissance['maximale'])
    rule2 = ctrl.Rule(temp_int_fuzzy['hypothermie'] & temp_ext_fuzzy['chaud'], puissance['moyenne'])
    rule3 = ctrl.Rule(temp_int_fuzzy['normal'] & temp_ext_fuzzy['chaud'], puissance['faible'])
    rule4 = ctrl.Rule(temp_int_fuzzy['normal'] & temp_ext_fuzzy['froid'], puissance['moyenne'])
    rule5 = ctrl.Rule(temp_int_fuzzy['fievre'] & temp_ext_fuzzy['chaud'], puissance['faible'])
    rule6 = ctrl.Rule(temp_int_fuzzy['fievre'] & temp_ext_fuzzy['froid'], puissance['moyenne'])

    #  CRÉATION DU SYSTÈME D'INFERENCE FLOUE

    systeme = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5, rule6])
    sim = ctrl.ControlSystemSimulation(systeme)

    #  INJECTION DES ENTRÉES ET CALCUL DE LA SORTIE

    sim.input['temp_ext'] = temp_ext
    sim.input['temp_int'] = temp_int
    sim.compute()

    puissance_val = sim.output['puissance']
    print(f"Puissance calculée : {puissance_val:.3f}\n")

    # SIMULATION DE L'ÉVOLUTION THERMIQUE DU CORPS

    erreur_totale = 0

    for i in range(100):
        temp_int = temp_int + (0.1 * puissance_val * (37 - temp_int)) / 100
        erreur = abs(temp_int - 37)
        erreur_totale += erreur

    # CALCUL DE LA FONCTION FITNESS

    fit = 1 / (1 + erreur_totale)
    print(f"Fitness calculée : {fit:.3f}\n")

    return fit

In [4]:
print(fitness(10,37))

Puissance calculée : 50.000

Fitness calculée : 1.000

1.0


In [5]:
! pip install PyGAD

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.6/89.6 kB 6.8 MB/s eta 0:00:00


In [6]:
import pygad

In [7]:
import random

# PARAMÈTRES

population_size = 30         # Population : 20-50
num_generations = 80         # Générations : 50-100
crossover_prob = 0.8         # 0.6 - 0.9
mutation_prob = 0.01         # 0.001 - 0.05
chromosome_length = 10       # Exemple : 10 gènes par individu

# SIMULATION DE LA TEMPÉRATURE

def simulate_temperature(Tin, Text, individu, steps=30):
    """
    Simule l'évolution de la température du patient.
    Modèle thermique simple :

    T(t+1) = T(t) + 0.1*puissance - 0.05*(T(t) - Text)

    La puissance est basée sur l'individu (ex : paramètres du régulateur flou).
    """
    power = sum(individu) / 10      # puissance entre 0 et 10 (normalisation)

    T = Tin
    error_sum = 0

    for _ in range(steps):
        # Évolution de la température
        T = T + 0.1 * power - 0.05 * (T - Text)
        error_sum += abs(T - 37)    # différence avec la température cible

    return error_sum

#  FITNESS

def fitness(individu, Tin, Text):
    """
    Fitness du TP :
    fitness = 1 / (1 + somme des erreurs)
    """
    error = simulate_temperature(Tin, Text, individu)
    return 1 / (1 + error)

# INITIALISATION DE LA POPULATION

def create_population(size, length):
    """
    Génère la population initiale d'individus aléatoires.
    Chaque gène = entier entre 0 et 10.
    """
    population = []
    for _ in range(size):
        individu = [random.randint(0, 10) for _ in range(length)]
        population.append(individu)
    return population

# SÉLECTION (Tournoi)

def selection(population, Tin, Text):
    """
    Sélection par tournoi : choisir 2 individus,
    garder celui qui a le meilleur fitness.
    """
    ind1 = random.choice(population)
    ind2 = random.choice(population)
    return ind1 if fitness(ind1, Tin, Text) > fitness(ind2, Tin, Text) else ind2

# CROSSOVER (1 point)

def crossover(parent1, parent2):
    """
    Croisement à un point :
    Combine parent1 et parent2 pour créer 2 enfants.
    """
    if random.random() < crossover_prob:
        point = random.randint(1, len(parent1)-1)
        child1 = parent1[:point] + parent2[point:]
        child2 = parent2[:point] + parent1[point:]
        return child1, child2
    else:
        return parent1[:], parent2[:]

# MUTATION

def mutate(individu):
    """
    Chaque gène peut muter avec probabilité mutation_prob.
    """
    for i in range(len(individu)):
        if random.random() < mutation_prob:
            individu[i] = random.randint(0, 10)
    return individu

# ALGORITHME GÉNÉTIQUE COMPLET

def genetic_algorithm(Tin, Text):
    """
    Exécute l'AG pour un scénario donné :
    - Tin : température interne initiale
    - Text : température externe
    """
    print("\n------------------------------------------------------")
    print(f"  Lancement de l'AG pour scénario : Tin={Tin}°C, Text={Text}°C")
    print("------------------------------------------------------")

    population = create_population(population_size, chromosome_length)

    for generation in range(num_generations):
        new_population = []

        while len(new_population) < population_size:

            # Sélection
            parent1 = selection(population, Tin, Text)
            parent2 = selection(population, Tin, Text)

            # Croisement
            child1, child2 = crossover(parent1, parent2)

            # Mutation
            child1 = mutate(child1)
            child2 = mutate(child2)

            # Ajouter les enfants
            new_population.extend([child1, child2])

        # Mise à jour de la population
        population = new_population[:population_size]

        # Affichage meilleur individu
        best = max(population, key=lambda ind: fitness(ind, Tin, Text))
        print(f"Génération {generation+1}: Fitness = {fitness(best, Tin, Text):.4f}, Individu = {best}")

    # Résultat final
    best = max(population, key=lambda ind: fitness(ind, Tin, Text))
    print("\n>>> Meilleur individu pour ce scénario :")
    print(f"Fitness = {fitness(best, Tin, Text):.4f}")
    print(f"Individu = {best}")

    return best

# TEST DE 3 SCÉNARIOS

# SCÉNARIO 1
genetic_algorithm(35, 10)

# SCÉNARIO 2
genetic_algorithm(37, 15)

# SCÉNARIO 3
genetic_algorithm(39, 25)



------------------------------------------------------
  Lancement de l'AG pour scénario : Tin=35°C, Text=10°C
------------------------------------------------------
Génération 1: Fitness = 0.0047, Individu = [10, 10, 8, 10, 5, 10, 4, 7, 2, 9]
Génération 2: Fitness = 0.0047, Individu = [10, 10, 8, 10, 5, 10, 4, 7, 2, 9]
Génération 3: Fitness = 0.0049, Individu = [8, 4, 9, 8, 8, 7, 8, 9, 9, 8]
Génération 4: Fitness = 0.0052, Individu = [10, 10, 7, 8, 8, 3, 8, 10, 8, 10]
Génération 5: Fitness = 0.0051, Individu = [8, 4, 9, 8, 8, 7, 8, 10, 8, 10]
Génération 6: Fitness = 0.0056, Individu = [10, 10, 9, 8, 8, 7, 8, 9, 9, 8]
Génération 7: Fitness = 0.0058, Individu = [10, 10, 9, 8, 8, 7, 10, 8, 8, 10]
Génération 8: Fitness = 0.0058, Individu = [10, 10, 9, 8, 8, 7, 10, 8, 8, 10]
Génération 9: Fitness = 0.0058, Individu = [10, 10, 9, 8, 8, 7, 8, 10, 8, 10]
Génération 10: Fitness = 0.0059, Individu = [10, 10, 9, 6, 8, 10, 8, 10, 8, 10]
Génération 11: Fitness = 0.0059, Individu = [10, 10, 9, 6, 

[2, 3, 7, 4, 2, 9, 6, 5, 10, 6]

## Optimisation avec Algorithme Génétique

On optimise ici les paramètres des fonctions d’appartenance de la température interne (normal)
Les 3 paramètres du trimf [a, b, c] seront optimisés.

Chromosome = [a, b, c]


In [8]:
def fitness_ga(chromosome, temp_ext_init, temp_int_init):
    a, b, c = chromosome

    temp_ext_fuzzy = ctrl.Antecedent(np.arange(0, 51, 1), 'temp_ext')
    temp_int_fuzzy = ctrl.Antecedent(np.arange(30, 42.1, 0.1), 'temp_int')
    puissance = ctrl.Consequent(np.arange(0, 101, 1), 'puissance')

    temp_ext_fuzzy['froid'] = fuzz.trapmf(temp_ext_fuzzy.universe, [0, 0, 10, 15])
    temp_ext_fuzzy['chaud'] = fuzz.trapmf(temp_ext_fuzzy.universe, [25, 30, 50, 50])

    temp_int_fuzzy['hypothermie'] = fuzz.trapmf(temp_int_fuzzy.universe, [30, 30, 35.5, 36.5])
    temp_int_fuzzy['normal'] = fuzz.trimf(temp_int_fuzzy.universe, [a, b, c])
    temp_int_fuzzy['fievre'] = fuzz.trapmf(temp_int_fuzzy.universe, [37.5, 38.5, 42, 42])

    puissance['faible'] = fuzz.trimf(puissance.universe, [0, 15, 30])
    puissance['moyenne'] = fuzz.trimf(puissance.universe, [30, 50, 70])
    puissance['maximale'] = fuzz.trimf(puissance.universe, [70, 85, 100])

    rule1 = ctrl.Rule(temp_int_fuzzy['hypothermie'] & temp_ext_fuzzy['froid'], puissance['maximale'])
    rule2 = ctrl.Rule(temp_int_fuzzy['hypothermie'] & temp_ext_fuzzy['chaud'], puissance['moyenne'])
    rule3 = ctrl.Rule(temp_int_fuzzy['normal'] & temp_ext_fuzzy['chaud'], puissance['faible'])
    rule4 = ctrl.Rule(temp_int_fuzzy['normal'] & temp_ext_fuzzy['froid'], puissance['moyenne'])
    rule5 = ctrl.Rule(temp_int_fuzzy['fievre'] & temp_ext_fuzzy['chaud'], puissance['faible'])
    rule6 = ctrl.Rule(temp_int_fuzzy['fievre'] & temp_ext_fuzzy['froid'], puissance['moyenne'])

    systeme = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5, rule6])
    sim = ctrl.ControlSystemSimulation(systeme)

    sim.input['temp_ext'] = temp_ext_init
    sim.input['temp_int'] = temp_int_init
    sim.compute()

    puissance_val = sim.output['puissance']

    temp_int = temp_int_init
    erreur_totale = 0

    for _ in range(100):
        temp_int = temp_int + (0.1 * puissance_val * (37 - temp_int)) / 100
        erreur_totale += abs(temp_int - 37)

    return 1 / (1 + erreur_totale)


In [9]:
# Paramètres AG
POP_SIZE = 30
GENERATIONS = 60
CROSS_RATE = 0.8
MUT_RATE = 0.02

# Initialisation population
population = np.random.uniform([35,36,37], [36,37,38], (POP_SIZE,3))

def select(pop, fitnesses):
    idx = np.random.choice(len(pop), p=fitnesses/fitnesses.sum())
    return pop[idx]

def crossover(parent1, parent2):
    if np.random.rand() < CROSS_RATE:
        point = np.random.randint(1,3)
        child = np.concatenate((parent1[:point], parent2[point:]))
        return child
    return parent1.copy()

def mutate(individual):
    for i in range(3):
        if np.random.rand() < MUT_RATE:
            individual[i] += np.random.uniform(-0.2,0.2)
    return individual

# Scénario test
temp_ext_init = 10
temp_int_init = 35

best_fitness_history = []

for gen in range(GENERATIONS):
    fitnesses = np.array([fitness_ga(ind, temp_ext_init, temp_int_init) for ind in population])

    new_population = []
    for _ in range(POP_SIZE):
        p1 = select(population, fitnesses)
        p2 = select(population, fitnesses)
        child = crossover(p1, p2)
        child = mutate(child)
        new_population.append(child)

    population = np.array(new_population)
    best_fitness_history.append(np.max(fitnesses))

best_index = np.argmax([fitness_ga(ind, temp_ext_init, temp_int_init) for ind in population])
best_solution = population[best_index]

print('Meilleur chromosome trouvé :', best_solution)
print('Fitness finale :', fitness_ga(best_solution, temp_ext_init, temp_int_init))


Meilleur chromosome trouvé : [35.76514662 36.5380934  37.75784132]
Fitness finale : 0.04439230733923799
